In [183]:
%pip install ipykernel
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install openpyxl
%pip install plotly
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import exp
import plotly.express as px
import PCP
#from functions import *


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Problema de una putisima puta
Considere una situación de una empresa que se dedica al procesamiento y entrega de productos derivados del acero para proyectos de construcción y ferreterías en la ciudad de Bogotá. La empresa recibe la información en pedidos llamados Remisiones, las cuales contiene la cantidad en kilogramos a procesar de diferentes productos. Cada remisión tiene una fecha de llegada en la cual el pedido puede empezar a ser procesado, las unidades de producto, la cantidad en kilogramos de producto, el tipo de producto (Figurado, Mallas electrosoldadas, y Materia Prima), y la descripción de cada producto especifico. Se tiene una información de pedidos recibidos en el mes de enero y febrero de 2011 en el anexo.<br>
La empresa tiene una capacidad de producción de 2600 kg/hora para el acero figurado, 800 kg/hora para las mallas electrosoldadas, y para la materia prima no se considera limitante dado que solo requiere alistamiento para despacho. La empresa no trabaja los días sábados, sin embargo un pedido que empiece a procesar el sábado y no se puede terminar, se retoma el lunes de manera inmediata. Como supuestos la empresa labora 24 horas al día de forma continua, los tiempos de alistamiento de los pedidos y transiciones se puede considerar despreciables para este caso, no existen precedencias entre los trabajos, y un trabajo que se programa debe terminarse por completo.<br>
La empresa desea realizar la programación semanal y diaria de los trabajos. Se le pide plantear diferentes enfoques de programación de los trabajos (remisiones) de acuerdo con la información presentada considerando $^1$:<br>
a) Minimizar el tiempo promedio de flujo<br>
b) Minimizar el total ponderado de los tiempos de terminación de los trabajos. $^2$<br>
c) Minimizar el total de anticipación y tardanza con una fecha común diaria de las 3:00 pm (la programación debe ser por día).<br>
d) Se tienen tres objetivos simultáneos: Minimizar el total ponderado de los tiempos de terminación de los trabajos, total ponderado de los tiempos de tardanza de los trabajos, y minimizar el total de trabajos tardíos $^3$. Se tiene una importancia relativa a la decisión de 80%, 10% y 10% respectivamente a cada uno de los objetivos. Compare diferentes reglas de secuenciación estáticas (al menos 8 reglas diferentes) y seleccione la mejor entre ellas para estos objetivos. En este caso se desea la programación semanal.<br>
e) Para el numeral anterior, considere que se puede aplicar reglas dinámicas para la programación, y se actualiza el programa cada 5 trabajos programados. Pruebe al menos 5 reglas dinámicas (debe investigar al menos 2 diferentes a las vistas en clase). En este caso se desea la programación semanal.<br>
f) Se requiere determinar las fechas de entrega de 4 grupos de trabajo por día. Debe plantear alternativas para el tamaño de los grupos. Se desea minimizar el costo total compuesto por tres componentes: uno asociado a la fecha de entrega con valor de $40.000/hora; uno de anticipación de $130.000/hora; y uno de tardanza de $200.000/hora.


# Base de datos
La base de datos proporcionada corresponde a una serie de remisiones, en donde se supuso que la fecha de llegada a la fabrica está marcada por la columna Fechas. a continuación se da información descriptiva de la base de datos. la cual ya estaba bien estructurada, por lo que no requirió ajustes adicionales.

In [184]:
DatosTaller = pd.read_excel('DatosTaller1.xlsx', sheet_name='Sheet1')
# drop nan columns and rows
DatosTaller = DatosTaller.dropna(
       axis=1, how='all').dropna(
       axis=0, how='all')
keys = DatosTaller.keys()
display(DatosTaller)
display(DatosTaller.describe())
print(keys)
for i in keys:
    display(DatosTaller[i].value_counts())
# Add a new column named tags with the 

,Nº REMISION,FECHA,Columna1,UND,KILOS,Tipo,DESCRIPCION,P - Horas,D -Dias,Columna2,Columna3,Columna5,Columna7,S,C
0,1000016974,2011-01-03,2011-01-03 15:00:00,57.4,57.4,Figurado,MAQUILA ACERO FIGURADO 5,00:01:19.477000,2011-01-04 15:00:00,2011-01-03 15:01:19.477,True,54,1,2011-01-03 15:00:00.000,2011-01-03 15:01:19.477
1,1000016974,2011-01-03,2011-01-03 15:00:00,1881.6,1881.6,Figurado,MAQUILA ACERO FIGURADO 8,00:43:25.292000,2011-01-04 15:00:00,2011-01-03 15:43:25.292,True,18,1,2011-01-03 15:01:19.477,2011-01-03 15:44:44.769
2,1000016974,2011-01-03,2011-01-03 15:00:00,3302.4,3302.4,Figurado,MAQUILA ACERO FIGURADO 7,01:16:12.554000,2011-01-04 15:00:00,2011-01-03 16:16:12.554,True,10,1,2011-01-03 15:44:44.769,2011-01-03 17:00:57.323
3,1000016974,2011-01-03,2011-01-03 15:00:00,3804.5,3804.5,Figurado,MAQUILA ACERO FIGURADO 6,01:27:47.769000,2011-01-04 15:00:00,2011-01-03 16:27:47.769,True,8,1,2011-01-03 17:00:57.323,2011-01-03 18:28:45.092
4,1000016975,2011-01-03,2011-01-03 15:00:00,648.0,648.0,Materia Prima,MAQUILA VARILLA RECTA 60 MIL N.6 X 12 MTS,00:00:00,2011-01-04 15:00:00,2011-01-03 15:00:00.000,True,58,1,2011-01-03 18:28:45.092,2011-01-03 18:28:45.092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2277,1000018134,2011-02-15,2011-02-15 15:00:00,144.0,144.0,Materia Prima,MAQUILA VARILLA RECTA 60 MIL N.8 X 12 MTS,00:00:00,2011-02-16 15:00:00,2011-02-15 15:00:00.000,True,43,2235,2011-03-18 15:17:37.472,2011-03-18 15:17:37.472
2278,1000018134,2011-02-15,2011-02-15 15:00:00,81.1,81.1,Figurado,MAQUILA ACERO FIGURADO 5,00:01:52.292000,2011-02-16 15:00:00,2011-02-15 15:01:52.292,True,35,2235,2011-03-18 15:17:37.472,2011-03-18 15:19:29.765
2279,1000018134,2011-02-15,2011-02-15 15:00:00,10855.6,10855.6,Figurado,MAQUILA ACERO FIGURADO 8,04:10:30.831000,2011-02-16 15:00:00,2011-02-15 19:10:30.831,True,1,2235,2011-03-18 15:19:29.765,2011-03-18 19:30:00.595
2280,1000018137,2011-02-15,2011-02-15 15:00:00,389.0,389.0,Figurado,MAQUILA ACERO FIGURADO 3,00:08:58.615000,2011-02-16 15:00:00,2011-02-15 15:08:58.615,True,21,2235,2011-03-18 19:30:00.595,2011-03-18 19:38:59.211


,Nº REMISION,UND,KILOS,Columna5,Columna7
count,2.282000e+03,2282.000000,2282.000000,2282.000000,2282.000000
mean,1.000018e+09,1547.885386,1971.994769,35.436459,1105.211218
std,3.398234e+02,3012.206607,4079.877369,24.897081,656.382837
min,1.000017e+09,0.000000,0.000000,1.000000,1.000000
25%,1.000017e+09,120.000000,179.225000,16.000000,545.000000
50%,1.000018e+09,457.000000,635.750000,32.000000,1113.000000
75%,1.000018e+09,1608.300000,1882.725000,51.000000,1684.000000
max,1.000018e+09,35549.000000,35610.000000,124.000000,2235.000000


Index(['Nº REMISION', 'FECHA', 'Columna1', 'UND', 'KILOS', 'Tipo',
       'DESCRIPCION', 'P - Horas', 'D -Dias', 'Columna2', 'Columna3',
       'Columna5', 'Columna7', 'S', 'C'],
      dtype='object')


1000018018    11
1000017223     9
1000017623     9
1000018024     9
1000017771     9
              ..
1000017662     1
1000017245     1
1000017240     1
1000017668     1
1000018143     1
Name: Nº REMISION, Length: 1111, dtype: int64

2011-02-02    143
2011-01-27    110
2011-02-07    109
2011-01-04     97
2011-02-10     92
2011-02-09     85
2011-01-28     85
2011-01-26     80
2011-01-13     78
2011-02-01     77
2011-01-05     73
2011-01-06     72
2011-01-25     71
2011-01-03     70
2011-01-17     67
2011-01-21     67
2011-01-24     66
2011-02-03     65
2011-01-29     64
2011-02-11     62
2011-02-04     58
2011-01-12     49
2011-02-15     48
2011-01-14     47
2011-01-19     46
2011-01-22     46
2011-02-05     44
2011-02-08     42
2011-01-15     41
2011-01-20     40
2011-02-14     38
2011-01-18     35
2011-01-11     27
2011-01-07     27
2011-01-31     25
2011-02-12     21
2011-01-23      9
2011-01-08      4
2011-01-30      2
Name: FECHA, dtype: int64

...

True    2282
Name: Columna3, dtype: int64

34     66
41     63
26     51
49     47
16     43
       ..
99      1
98      1
94      1
122     1
101     1
Name: Columna5, Length: 123, dtype: int64

1476    143
1113    110
1786    109
71       97
2022     92
1937     85
1223     85
1033     80
420      78
1399     77
168      73
241      72
962      71
1        70
586      67
774      67
896      66
1619     65
1308     64
2114     62
1684     58
371      49
2235     48
498      47
688      46
841      46
1742     44
1895     42
545      41
734      40
2197     38
653      35
344      27
313      27
1374     25
2176     21
887       9
340       4
1372      2
Name: Columna7, dtype: int64

2011-02-02 16:49:06.140    17
2011-01-18 19:41:53.752     9
2011-02-03 02:45:29.170     8
2011-02-25 11:41:24.779     8
2011-02-04 11:24:19.340     7
                           ..
2011-02-01 11:36:26.263     1
2011-02-01 09:10:01.155     1
2011-02-01 08:58:44.632     1
2011-02-01 08:51:32.632     1
2011-03-18 19:38:59.211     1
Name: S, Length: 1779, dtype: int64

2011-02-02 16:49:06.140    17
2011-01-18 19:41:53.752     9
2011-02-03 02:45:29.170     8
2011-02-25 11:41:24.779     8
2011-02-04 11:24:19.340     7
                           ..
2011-02-01 16:30:43.155     1
2011-02-01 11:36:26.263     1
2011-02-01 09:10:01.155     1
2011-02-01 08:58:44.632     1
2011-03-18 23:40:27.826     1
Name: C, Length: 1779, dtype: int64

# Preparación de Datos

Para trabajar los datos como un problema de Scheduling para una máquina se identifica la necesidad de los siguientes datos:
1. *j*, que es la etiqueta de cada trabajo o el indicador del mismo, en donde j va desde 1 hasta n.
2. *d*, que es la fecha en la que se promete entregar el trabajo.
3. *p*, que es la cantidad de tiempo que toma procesar un trabajo o remisión.
4. ***r***, que es la fecha en la que se espera recibir el trabajo o remisión.
finalmente *m* es 1, ya que se trata la fabrica como una máquina que opera todos los trabajos.


Entonces,
*j* en la base de datos es información de etiquetas de las remisiones o trabajos.
*r* fecha de llegada de los trabajos a procesar identificada por la columna 'FECHA'.
*d* es la fecha de llegada más dos días $d_j=r_j+48 \space horas$.
*p* depende de 'KILOS' y 'Tipos', en donde:
$$'KILOS'/2600Kg/h \space if 'Tipos'='Figurado''KILOS'/800Kg/h\space if 'Tipos'='Mallas'0 if \space 'Tipos'='Materia Prima'$$


Importante, a partir de este punto se considerará que el tiempo del domingo no existe para nada más que determinar el due date. por lo que se linealizarán todos los tiempos para que inicien en min(r) y que todas las fechas sean numeros y sean x-min(r)


In [185]:
def get_weekday(row, column = 'FECHA'):
    """
    function that takes a row and returns the day of the week
    """
    # get the date on the row
    date = row[column]
    # get the day of the week
    weekday = date.day_name()
    return weekday
def get_r(row, min_date):
    """
    function that takes a date and returns it to hours ignoring sundays"""
    # get the date on the row
    date = row['FECHA']
    # get the number of days since the minimum date
    diff = date - min_date
    days = diff.days
    # get the number of weeks since the minimum date
    weeks = days // 7
    # get the number of days since the minimum date ignoring the weekends
    days = days - weeks
    # get the number of hours since the minimum date ignoring the weekends
    hours = days*24
    return days
def get_d(row, n_days):
    """
    function that takes a row and returns r_{j} and adds n days to r_{j}
    """
    # get the number of hours since the minimum date ignoring the weekends
    r = row['r']
    # add n hours to r_{j}
    r = r + n_days
    return r
def get_p(row):
    """
    function that evaluates when a row and creates a column based on the
    value of 'Tipo' where:
    if the value is 'Figurado' it returns 'KILOS'/ 2600,
    if the value is 'Mallas' it returns 'KILOS'/ 800, and
    if the value is 'Materia Prima' it returns 0
    """
    if row['Tipo'] == 'Figurado':
        return row['KILOS']/(2600*24)
    elif row['Tipo'] == 'Mallas':
        return row['KILOS']/(800*24)
    else:
        return 0
def get_tags(row):
    """
    function that takes the values of the columns
    'Nº REMISION', 'FECHA', 'UND', 'KILOS', 'Tipo', 'DESCRIPCION', 'weekday'
    and returns a string with the values separated by a comma
    """
    # get the values of the columns
    n_remision = row['Nº REMISION']
    fecha = row['FECHA']
    und = row['UND']
    kilos = row['KILOS']
    tipo = row['Tipo']
    descripcion = row['DESCRIPCION']
    weekday = row['weekday']
    # create the string
    tags = f'{n_remision}, {fecha}, {und}, {kilos}, {tipo}, {descripcion}, {weekday}'
    return tags
def float_to_date_2(list, start_date):
    """"
    function that takes a list of floats that represent days and returns a list of dates
    """
    dates = []
    for f in list:
        date = start_date + datetime.timedelta(days=int(f))
        dates.append(date)
    return dates
def convert_floats_to_dates(float_list, start_date):
    dates = []
    for f in float_list:
        date = start_date + datetime.timedelta(days=int(f))
        if date.weekday() != 6:  # 6 is Sunday
            dates.append(date)
    return dates
def float_days_to_timeformat(hours):
    """
    function that takes a list of floats that represent days and returns a list of strings in the format 'HH:MM'
    """
    # create a list of strings
    timeformat = []
    # iterate over the list of floats
    for h in hours:
        # get the number of weeks
        weeks = h // 7
        # get the number of days
        days = h - weeks
        # get the number of hours
        hours = days*24
        # get the number of minutes
        minutes = (hours - int(hours))*60
        # create a string
        string = f'{int(hours)}:{int(minutes)}'
        # append the string to the list of strings
        timeformat.append(string)
    return timeformat
# create a new column 'weekday' and use the function get_weekday to fill it
DatosTaller['weekday'] = DatosTaller.apply(get_weekday, axis=1)
# apply the function get_r to the column 'FECHA'
DatosTaller['r'] = DatosTaller.apply(get_r, axis=1, args=(DatosTaller['FECHA'].min(),))
# apply the function get_d to the column 'r'
DatosTaller['d'] = DatosTaller.apply(get_d, axis=1, args=(1,))
# apply the function get_p to the column 'Tipo'
DatosTaller['p'] = DatosTaller.apply(get_p, axis=1)
# apply the function get_tags to the columns 'Nº REMISION', 'FECHA', 'UND', 'KILOS', 'Tipo', 'DESCRIPCION'
DatosTaller['tags'] = DatosTaller.apply(get_tags, axis=1)

## 1) Minimizar el tiempo de promedio de flujo

El tiempo promedio del flujo se minimiza con la regla de asignación SPT (shortest processing time)

In [186]:
# n: numero de trabajos
n = len(DatosTaller)
# p: lista de tiempos de procesamiento
p = DatosTaller['p'].tolist()
# d: lista de tiempos de entrega
d = DatosTaller['d'].tolist()
# r: lista de tiempos de inicio
r = DatosTaller['r'].tolist()
# tags: lista de etiquetas
tags = DatosTaller['tags'].tolist()
# Create a Single Machine instance
SM = PCP.SingleMachine(n=n, p=p, d=d, r=r)
SM.J = tags
# Solve by SPT rule
SM.SPT()
# Get the mean flow time
print(sum(SM.C)/SM.n)

1000016975, 2011-01-03 00:00:00, 1175.0, 1175.0, Materia Prima, MAQUILA VARILLA RECTA 60 MIL N.7 X 12 MTS, Monday :  0 0 -> 0 - 0.0 = 0.0
1000016975, 2011-01-03 00:00:00, 648.0, 648.0, Materia Prima, MAQUILA VARILLA RECTA 60 MIL N.6 X 12 MTS, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016976, 2011-01-03 00:00:00, 309.1, 309.1, Materia Prima, MAQUILA VARILLA RECTA 60 MIL N.3 X 12 MTS, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016977, 2011-01-03 00:00:00, 309.1, 309.1, Materia Prima, MAQUILA VARILLA RECTA 60 MIL N.3 X 12 MTS, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016982, 2011-01-03 00:00:00, 108.0, 108.0, Materia Prima, MAQUILA VARILLA RECTA 60 MIL N.4 X 12 MTS, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016989, 2011-01-03 00:00:00, 100.0, 100.0, Materia Prima, MAQUILA ALAMBRE RECOCIDO CAL 18, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016990, 2011-01-03 00:00:00, 250.0, 250.0, Materia Prima, MAQUILA ALAMBRE RECOCIDO CAL 18, Monday :  0 0.0 -> 0 - 0.0 = 0.0
1000016991, 2011-01-03 00:00:00, 322.6, 322.6, Materia

In [187]:
start_date = DatosTaller['FECHA'].min()
results = pd.DataFrame({'Task': SM.J,
                          'p': float_days_to_timeformat(SM.p),
                          'd': SM.d,
                          'r': SM.r,
                          'Start': float_to_date_2(SM.S, start_date),
                          'Finish': float_to_date_2(SM.C, start_date),
                          'L': SM.L,
                          'T': SM.T,
                          'E': SM.E})
# Add a new column named 'M' with a value of 1
results['M'] = 1
# make a weekday column with the 'Start' of result
results['weekday'] = results.apply(get_weekday, axis=1, args=('Start',))
fig = px.timeline(results,x_start='Start',
                  x_end='Finish',
                  y='Task',
                  color='M',
#                  color_continuous_scale=["blue",
#                                          "red",
#                                          "green",
#                                          "yellow",
#                                          "grey"]
)
fig.show(renderer="browser")

## 2) Minimizar el total ponderado de los tiempos de terminación de los trabajos
Se realiza con un WSPT(/)

In [188]:
Weight_data = DatosTaller.groupby('Nº REMISION').nunique()['Tipo']
DatosTaller = pd.merge(DatosTaller, Weight_data, on='Nº REMISION')
DatosTaller = DatosTaller.rename(columns={'Tipo_y': 'Weight', 'Tipo_x': 'Tipo'})
W = DatosTaller['Weight'].tolist()
SM = PCP.SingleMachine(n=n, p=p, d=d, r=r)
SM.W = W
SM.J = tags
SM.WSPT()
print(sum(SM.C)/SM.n)
display(DatosTaller)
start_date = DatosTaller['FECHA'].min()
results = pd.DataFrame({'Task': SM.J,
                          'p': float_days_to_timeformat(SM.p),
                          'd': SM.d,
                          'r': SM.r,
                          'Start': float_to_date_2(SM.S, start_date),
                          'Finish': float_to_date_2(SM.C, start_date),
                          'L': SM.L,
                          'T': SM.T,
                          'E': SM.E})
# Add a new column named 'M' with a value of 1
results['M'] = 1
# make a weekday column with the 'Start' of result
results['weekday'] = results.apply(get_weekday, axis=1, args=('Start',))
fig = px.timeline(results,x_start='Start',
                  x_end='Finish',
                  y='Task',
                  color='M',
#                  color_continuous_scale=["blue",
#                                          "red",
#                                          "green",
#                                          "yellow",
#                                          "grey"]
)
fig.show(renderer="browser")

AttributeError: 'SingleMachine' object has no attribute 'WSPT'